<a href="https://colab.research.google.com/github/NeerajMehta15/Fine_Tuning_LLMs/blob/main/Fine_Tuning_LLaMA_2_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%pip install accelerate peft bitsandbytes transformers trl

In [ ]:
!pip install --upgrade accelerate transformers

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer
from accelerate import Accelerator

In [ ]:
# Model from Hugging Face hub
base_model = "NousResearch/Llama-2-7b-chat-hf"
# New instruction dataset
guanaco_dataset = "mlabonne/guanaco-llama2-1k"
# # Fine-tuned model
new_model = "/content/new_model"

In [ ]:
dataset = load_dataset(guanaco_dataset, split="train")

In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}                         #Loaded in GPU
)
model.config.use_cache = False
model.config.pretraining_tp = 1                 # No tensor parallelism is used; the model runs on a single device

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    output_dir="/content/results",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
accelerator = Accelerator()

In [ ]:
trainer = SFTTrainer(
    model=accelerator.prepare(model), # Preparing the model with the accelerator
    train_dataset=accelerator.prepare(dataset), # Prepare the dataset with the accelerator
    peft_config=peft_params,
    tokenizer=tokenizer,
    args=training_params,
)


In [ ]:
trainer.train()

In [ ]:
trainer.model.save_pretrained(new_model)

if trainer.processing_class:
    trainer.processing_class.save_pretrained(new_model)


In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "What are the ingredients of Dal Makhni?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
prompt = "What about palak paneer?"
result = pipe(prompt)
print(result[0]['generated_text'])